# Comp 472 Mini-Project 2

## 1. Game Setup

In [1]:
import pprint
from queue import PriorityQueue
import copy
import time

In [2]:
# Definition of car object 
class Car:
    def __init__(self, name, fuel, coordinates, orientation):
        
        self.name = name
        self.fuel = fuel
        self.coordinates = coordinates # list of coordinates that represents its position in the board
        self.orientation = orientation
        
    # Function used to print the information of the car
    def carInfo(self):
        print("Name: ", self.name, ", Fuel: ",self.fuel ,", Coordinates: ", self.coordinates, ", Orientation: ", self.orientation)

In [3]:
class Successor:
    def __init__(self, carName, carDirection, newPosition):
        self.carName = carName
        self.carDirection = carDirection
        self.newPosition = newPosition

In [4]:
# Possible letters that could represent cars in the board
carLetters =   ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

# Definition of board object
class Board:
    def __init__(self, puzzleLine):
        
        self.board = []
        self.cars = {} # dictionary of car objects present in the board
        self.dimension = 6 # dimension
        self.puzzleLine = puzzleLine

        # Creating empty board
        boardMatrix = [[0 for x in range(self.dimension)] for x in range(self.dimension)] 

        # Filling board with puzzle line information (36 characters)
        a = 0    
        for i in range(self.dimension):
            for j in range(self.dimension):
                boardMatrix[i][j] = puzzleLine[a]
                a += 1
        
        # Assign board matrix to board object
        self.board = boardMatrix

        # Getting coordinates of the cars in the board
        coordinatesDict = {}
        for letter in carLetters:
            if letter in self.puzzleLine: # If car letter exists in the board
                coordinatesDict[letter] = [[x, y] for x, li in enumerate(self.board) for y, val in enumerate(li) if val==letter]

        # Checking for initial fuel units for the cars in the board for the specific puzzle (Default fuel units = 100)
        fuelDict = {}
        fuelInfoFromPuzzleLine = self.puzzleLine[36:].strip()
        if fuelInfoFromPuzzleLine != "": # If additional info exists after the puzzle line, then initial fuel info exists
            initialFuelInfo = fuelInfoFromPuzzleLine.split()
            for fuel in initialFuelInfo:
                fuelDict[fuel[0]] = int(fuel[1:])

        # Creating car object with obtained information
        for carName in coordinatesDict:

            # Checking the orientation of each car in the board
            # If the x of the coordinates are equal, then horizontal. If not, then vertical
            if coordinatesDict[carName][0][0] == coordinatesDict[carName][1][0]:
                orientation = "horizontal"
            else:
                orientation = "vertical"

            if carName in fuelDict:
                self.cars[carName] = Car(carName, fuelDict[carName], coordinatesDict[carName], orientation)
            else:
                self.cars[carName] = Car(carName, 100, coordinatesDict[carName], orientation)


    # Function to print the initial board matrix            
    def printInitialBoard(self):
        a = 0
        for i in range(self.dimension):
            print(self.puzzleLine[a:(a+6)], " ")
            a += 6
    
    # Function to print the current board
    def printBoardMatrix(self):
        pprint.pprint(self.board)
        
        
    # Function to generate the possible states that the board can be in depending on the possible moves of one car    
    def generateSuccessorStates(self):
        successors = [] # List of successor objects (each contains info needed for the moveCar function)

        for car in self.cars.values():

            # horizontal -- ======================
            if car.orientation == "horizontal":
                
                # Check left direction ===========
                left = car.coordinates[0][:] # copy most left coordinate of this car

                while(left[1] > 0): # until we reach left edge of board
                    left[1] -= 1 # check one cell to the left
                    
                    if self.board[left[0]][left[1]] == '.': # if that cell is empty
                        successors.append(Successor(car.name, "Left", left[:]))
                    else:
                        break

                # Check right direction ===========
                right = car.coordinates[-1][:] # copy most right coordinate of this car

                while(right[1] < 5): # until we reach right edge of board
                    right[1] += 1 # check one cell to right

                    if self.board[right[0]][right[1]] == '.':
                        successors.append(Successor(car.name, "Right", right[:]))
                    else:
                        break

            # vertical | ======================
            if car.orientation == "vertical":
                
                # Check up direction ===========
                up = car.coordinates[0][:]
                
                while(up[0] > 0):
                    up[0] -= 1
                    if self.board[up[0]][up[1]] == '.':
                        successors.append(Successor(car.name, "Up", up[:]))
                    else:
                        break

                # Check down direction ===========
                down = car.coordinates[-1][:]
                
                while(down[0] < 5):
                    down[0] += 1
                    if self.board[down[0]][down[1]] == '.':
                        successors.append(Successor(car.name, "Down", down[:]))
                    else:
                        break
            
        return successors
    
    # helper function (used inside moveCar function)
    def updateBoard(self, carName, coordinates, newlyOccupied):
        if newlyOccupied:
            self.board[coordinates[0]][coordinates[1]] = carName
        else:
            self.board[coordinates[0]][coordinates[1]] = '.'
    
    #
    def moveCarOnBoard(self, carName, carDirection, newPosition):
        
        target_car = self.cars[carName]
        positions_moved = 0
        
        if target_car.orientation == "vertical":
            if carDirection == "Up":
                
                for i in range(len(target_car.coordinates)):
                    if i < 1:
#                         positions_moved = target_car.coordinates[0][:][0] - newPosition[0]
                        self.updateBoard(carName, target_car.coordinates[0][:], False)
#                         target_car.coordinates[0][:] = newPosition
#                         target_car.fuel -= positions_moved
                        self.updateBoard(carName, newPosition, True)
                    else:
                        carCoordinate = target_car.coordinates[i]
                        self.updateBoard(carName, carCoordinate, False)
                        carCoordinate[0] = newPosition[0]+i
                        self.updateBoard(carName, carCoordinate, True)

            elif carDirection == "Down":
                for i in range(len(target_car.coordinates)):
                    if i < 1:
#                         positions_moved = newPosition[0] - target_car.coordinates[-1][:][0]
                        self.updateBoard(carName, target_car.coordinates[-1][:], False)
#                         target_car.coordinates[-1][:] = newPosition
#                         target_car.fuel -= positions_moved
                        self.updateBoard(carName, newPosition, True)
                    else:
                        carCoordinate = target_car.coordinates[i-1]
                        self.updateBoard(carName, carCoordinate, False)
                        carCoordinate[0] = newPosition[0]-i
                        self.updateBoard(carName, carCoordinate, True)

                
        elif target_car.orientation == "horizontal":
            if carDirection == "Left":
                
                for i in range(len(target_car.coordinates)):
                    if i < 1:
#                         positions_moved = target_car.coordinates[0][:][1] - newPosition[1] 
                        self.updateBoard(carName, target_car.coordinates[0][:], False)
#                         target_car.coordinates[0][:] = newPosition
#                         target_car.fuel -= positions_moved
                        self.updateBoard(carName, newPosition, True)
                    else:
                        carCoordinate = target_car.coordinates[i]
                        self.updateBoard(carName, carCoordinate, False)
                        carCoordinate[1] = newPosition[1]+i
                        self.updateBoard(carName, carCoordinate, True)
            
            elif carDirection == "Right":
                
                for i in range(len(target_car.coordinates)):
                    if i < 1:
#                         positions_moved = newPosition[1] - target_car.coordinates[-1][:][1]
                        self.updateBoard(carName, target_car.coordinates[-1][:], False)
#                         target_car.coordinates[-1][:] = newPosition
#                         target_car.fuel -= positions_moved
                        self.updateBoard(carName, newPosition, True)
                    else:
                        carCoordinate = target_car.coordinates[i-1]
                        self.updateBoard(carName, carCoordinate, False)
                        carCoordinate[1] = newPosition[1]-i
                        self.updateBoard(carName, carCoordinate, True)
    
    #
    def updateCar(self, carName, carDirection, newPosition):
        
        target_car = self.cars[carName]
        positions_moved = 0
        
        if target_car.orientation == "vertical":
            if carDirection == "Up":
                
                for i in range(len(target_car.coordinates)):
                    if i < 1:
                        positions_moved = target_car.coordinates[0][:][0] - newPosition[0]
#                         self.updateBoard(carName, target_car.coordinates[0][:], False)
                        target_car.coordinates[0][:] = newPosition
                        target_car.fuel -= positions_moved
#                         self.updateBoard(carName, newPosition, True)
                    else:
#                         self.updateBoard(carName, target_car.coordinates[i], False)
                        target_car.coordinates[i][0] = newPosition[0]+i
#                         self.updateBoard(carName, target_car.coordinates[i], True)

            elif carDirection == "Down":
                for i in range(len(target_car.coordinates)):
                    if i < 1:
                        positions_moved = newPosition[0] - target_car.coordinates[-1][:][0]
#                         self.updateBoard(carName, target_car.coordinates[-1][:], False)
                        target_car.coordinates[-1][:] = newPosition
                        target_car.fuel -= positions_moved
#                         self.updateBoard(carName, newPosition, True)
                    else:
#                         self.updateBoard(carName, target_car.coordinates[i-1], False)
                        target_car.coordinates[i-1][0] = newPosition[0]-i
#                         self.updateBoard(carName, target_car.coordinates[i-1], True)

                
        elif target_car.orientation == "horizontal":
            if carDirection == "Left":
                
                for i in range(len(target_car.coordinates)):
                    if i < 1:
                        positions_moved = target_car.coordinates[0][:][1] - newPosition[1] 
#                         self.updateBoard(carName, target_car.coordinates[0][:], False)
                        target_car.coordinates[0][:] = newPosition
                        target_car.fuel -= positions_moved
#                         self.updateBoard(carName, newPosition, True)
                    else:
#                         self.updateBoard(carName, target_car.coordinates[i], False)
                        target_car.coordinates[i][1] = newPosition[1]+i
#                         self.updateBoard(carName, target_car.coordinates[i], True)
            
            elif carDirection == "Right":
                
                for i in range(len(target_car.coordinates)):
                    if i < 1:
                        positions_moved = newPosition[1] - target_car.coordinates[-1][:][1]
#                         self.updateBoard(carName, target_car.coordinates[-1][:], False)
                        target_car.coordinates[-1][:] = newPosition
                        target_car.fuel -= positions_moved
#                         self.updateBoard(carName, newPosition, True)
                    else:
#                         self.updateBoard(carName, target_car.coordinates[i-1], False)
                        target_car.coordinates[i-1][1] = newPosition[1]-i
#                         self.updateBoard(carName, target_car.coordinates[i-1], True)

    def moveCar(self, carName, carDirection, newPosition):
        
        target_car = self.cars[carName]
        positions_moved = 0
        
        if target_car.orientation == "vertical":
            if carDirection == "Up":
                
                for i in range(len(target_car.coordinates)):
                    if i < 1:
                        positions_moved = target_car.coordinates[0][:][0] - newPosition[0]
                        self.updateBoard(carName, target_car.coordinates[0][:], False)
                        target_car.coordinates[0][:] = newPosition
                        target_car.fuel -= positions_moved
                        self.updateBoard(carName, newPosition, True)
                    else:
                        self.updateBoard(carName, target_car.coordinates[i], False)
                        target_car.coordinates[i][0] = newPosition[0]+i
                        self.updateBoard(carName, target_car.coordinates[i], True)

            elif carDirection == "Down":
                for i in range(len(target_car.coordinates)):
                    if i < 1:
                        positions_moved = newPosition[0] - target_car.coordinates[-1][:][0]
                        self.updateBoard(carName, target_car.coordinates[-1][:], False)
                        target_car.coordinates[-1][:] = newPosition
                        target_car.fuel -= positions_moved
                        self.updateBoard(carName, newPosition, True)
                    else:
                        self.updateBoard(carName, target_car.coordinates[i-1], False)
                        target_car.coordinates[i-1][0] = newPosition[0]-i
                        self.updateBoard(carName, target_car.coordinates[i-1], True)

                
        elif target_car.orientation == "horizontal":
            if carDirection == "Left":
                
                for i in range(len(target_car.coordinates)):
                    if i < 1:
                        positions_moved = target_car.coordinates[0][:][1] - newPosition[1] 
                        self.updateBoard(carName, target_car.coordinates[0][:], False)
                        target_car.coordinates[0][:] = newPosition
                        target_car.fuel -= positions_moved
                        self.updateBoard(carName, newPosition, True)
                    else:
                        self.updateBoard(carName, target_car.coordinates[i], False)
                        target_car.coordinates[i][1] = newPosition[1]+i
                        self.updateBoard(carName, target_car.coordinates[i], True)
            
            elif carDirection == "Right":
                
                for i in range(len(target_car.coordinates)):
                    if i < 1:
                        positions_moved = newPosition[1] - target_car.coordinates[-1][:][1]
                        self.updateBoard(carName, target_car.coordinates[-1][:], False)
                        target_car.coordinates[-1][:] = newPosition
                        target_car.fuel -= positions_moved
                        self.updateBoard(carName, newPosition, True)
                    else:
                        self.updateBoard(carName, target_car.coordinates[i-1], False)
                        target_car.coordinates[i-1][1] = newPosition[1]-i
                        self.updateBoard(carName, target_car.coordinates[i-1], True)
    
    # 
    def getSuccessorString(self):
        successorStr = ""
        for line in self.board:
            for char in line:
                successorStr += char
                
        for car in self.cars.values():
            if car.fuel < 100:
                successorStr += (" " + car.name + str(car.fuel)) 
        return successorStr
         
    # Function that checks if the car is A    
    def isGoal(self):
        
        Ambulance = self.cars['A']
        if Ambulance.coordinates[-1] == [2,5]:
            return True
        else:
            return False
    
    # Function that checks if the car is at the exit
    def isExit(self, carName):
        car = self.cars[carName]

        # coordinates should also be updated
        # remove car from coordinatesDict
        if car.orientation == "horizontal":
            if car.coordinates[-1] == [2,5]:
                # exit
                for each in car.coordinates:
                    self.board[each[0]][each[1]] = '.'   
            self.cars.pop(carName)
        else:
            pass

        return False

In [5]:
# Definition of State class that stores the board and its parent, and g(n), h(n), f(n)
class State:
    def __init__(self, parent, board, cost, heuristic, f_n):
        self.parent = parent
        self.board = board
        self.cost = cost
        self.heuristic = heuristic
        self.f_n = f_n
        
    def printState(self):
#         print("Parent Board: ", self.parent.printBoardMatrix())
#         print("Board: ", self.board.printBoardMatrix())
        print("Puzzle Line: ", self.board.puzzleLine)
        print("Cost: ", self.cost)

In [6]:
# Current logic:
# - Initial state -> generate successors -> pick a successor and move it
# - Then generate successors

# We need:

# - a function that concatenates the board matrix characters into a string to put it in the output file

# - a function that checks if the successor state is in the CLOSED list or not, if it isn't then we must add it to the OPEN list.
# - Then when we pop a state from the OPEN list, we will use the isGoal function to check if it is goal space or not. If it isn't, then we will apply the moveCar function to move the car and update the coordinates of the car and the board. Then, we will apply the isExit function to check if it reached the exit, if it did we remove it and update board and coordinates. If it did not, we move on and generate the next successors. Add to CLOSED and OPEN list as before.
# - Perhaps we need a function to add states to whichever list it needs to be in
 
# - Maybe in this case, we put isGoal and isExit together?

# - Seems like we have to change our successors function... or maybe not

# - Gotta add a function to generate a single spreadsheet for the 50 puzzles and a function to generate 50 random puzzles. Only need 1 file of 50 random puzzles and this is what we will wokr with.

# Questions: 

# - How does the cost ever differ if at every level of the "tree", the cars only make 1 move which is equivalent to 1 cost? So in UCS, wouldn't every state only be 1 cost? How would that be the lowest cost solution? In the end, it is a BFS!
# - Same question as Paul regarding the 50 random puzzles - just the spreadsheet
# - Generate new boards? Preferably yes, since it would be tough when we are backtracking to a previous state (we'll have to check it out) (Check with Deokyeong)

In [7]:
# Proposed Logic:
    
# - From the successor states, we create a board for each, calculate the cost and make a State object which will be stored in
# OPEN (Python has a priority queue) AND CLOSED lists
# - When we have chosen a STATE and added to our OPEN list, we will move the cars and update the car coordinates 

In [8]:
file_path = "SampleInputOutput/Sample/sample-input.txt"

def readPuzzles(file_path):
    hashtag = "#"
    with open(file_path) as file:
        puzzles = [line.rstrip() for line in file]
        puzzles = list(filter(None, puzzles))
        
        for line in puzzles.copy():
            if hashtag in line:
                puzzles.remove(line)
    return puzzles  

In [9]:
Boards = []
def printAllInfo(file_path):
    a=1
    lines = readPuzzles(file_path) # extract lines from input file
    
    
    
    for each in lines: # for each board, print infos
        print("--Board ",a,"--")
        test = Board(each)
        Boards.append(test)
        test.printInitialBoard()
        print("\n>>Successors: ")
        pprint.pprint(test.generateSuccessorStates())
        print("\n>>Car info")
        for car in test.cars.values():
            car.carInfo()
        print("==================================\n")
        a+=1

In [10]:
printAllInfo(file_path)

--Board  1 --
BBIJ..  
..IJCC  
..IAAM  
GDDK.M  
GH.KL.  
GHFFL.  

>>Successors: 

>>Car info
Name:  A , Fuel:  100 , Coordinates:  [[2, 3], [2, 4]] , Orientation:  horizontal
Name:  B , Fuel:  100 , Coordinates:  [[0, 0], [0, 1]] , Orientation:  horizontal
Name:  C , Fuel:  100 , Coordinates:  [[1, 4], [1, 5]] , Orientation:  horizontal
Name:  D , Fuel:  100 , Coordinates:  [[3, 1], [3, 2]] , Orientation:  horizontal
Name:  F , Fuel:  100 , Coordinates:  [[5, 2], [5, 3]] , Orientation:  horizontal
Name:  G , Fuel:  100 , Coordinates:  [[3, 0], [4, 0], [5, 0]] , Orientation:  vertical
Name:  H , Fuel:  100 , Coordinates:  [[4, 1], [5, 1]] , Orientation:  vertical
Name:  I , Fuel:  100 , Coordinates:  [[0, 2], [1, 2], [2, 2]] , Orientation:  vertical
Name:  J , Fuel:  100 , Coordinates:  [[0, 3], [1, 3]] , Orientation:  vertical
Name:  K , Fuel:  100 , Coordinates:  [[3, 3], [4, 3]] , Orientation:  vertical
Name:  L , Fuel:  100 , Coordinates:  [[4, 4], [5, 4]] , Orientation:  vertica

## 2.1 State Space Search

### 2.1.1 Uniform Cost Search

In [11]:
# def findSolutionPath(rootState, CLOSED):
#     solutionPathIndexes = []
    
#     goalState = CLOSED[-1]
    
# #     x = list(reversed(CLOSED))
# #     for i in x:
# #         i.printState()
        
#     for 
    
#     while True:
        
#         if goalState.board == rootState.board:
#             break
        
#         for state in CLOSED:
            
#             if state == rootState:
#                 continue
            
#             if goalState.parent.board == :
#                 solutionPathIndexes.append(state)
#                 goalState = state
# #                 print(goalState.printState())
# #                 print(state.printState())
#                 break
    
#     return solutionPath 

In [12]:
def UniformCostSearch(Root):
    CLOSED = []
    OPEN = []
    
    # Creating the initial State
    # State(parent, board, cost, heuristic, f_n)
    rootState = State(None, Root, 0, 0, 0)
    
    # Append the rootState (initial state) to the OPEN list
    OPEN.append(rootState)
    
    # Check if the rootState is goal
    if rootState.board.isGoal():
        return True
    
    while True:
        
        # If the OPEN list is empty, then there is no solution found
        if not OPEN:
            print("No Solution found.")
            break
        
        # List of strings of the generated successor boards
        SuccessorStrings = []
        
        # "Visit" the first state in the OPEN list and add it to the CLOSED list
        parentState = OPEN.pop(0)
        CLOSED.append(parentState)
        
        # Checking if the current state we are visiting is the goal state
        if (parentState.board).isGoal():
            # TO-DO!!: AFTER FINDING THE GOAL STATE WE HAVE TO BACKTRACK AND FIND SOLUTION PATH
            # CURRENTLY ONLY TEMPORARILY PRINTING OPEN CLOSED LISTS
            print("CLOSED List: ")
            for state in CLOSED:
                print("\n")
                print(state.printState())
            print("\nOPEN List: ")
            for state in OPEN:
                print("\n")
                print(state.printState())
#             solutionPath = findSolutionPath(rootState, CLOSED)
#             for state in solutionPath:
#                 state.printState()
            break
        
        # Make a copy of the parent state as we will use the copy to generate the strings of the possible successor boards
        possibleNewState = copy.deepcopy(parentState)
        
        # Generate possible successor moves for the cars on the board
        successors = (parentState.board).generateSuccessorStates()
        
        
        # Create list of strings for each possible successor board
        for infoObj in successors:
            # moveCar(self, carName, carDirection, newPosition)
            (possibleNewState.board).moveCar(infoObj.carName, infoObj.carDirection, infoObj.newPosition)
            
            # getSuccessorString()
            SuccessorStrings.append((possibleNewState.board).getSuccessorString())
            
            # Reset the board to parent state to apply the moveCar function again
            possibleNewState = copy.deepcopy(parentState)

            
        # Create board object / and states at the same time
        for eachStr in SuccessorStrings:
            
            # State( parentBoard, board, cost, heuristic, f_n):
            oneStateObj = State(parentState.board, Board(eachStr), (parentState.cost+1), 0, (parentState.cost+1))
               
            # Checking if the current successor board is already present in the CLOSED list
            if(not any(state.board.board == oneStateObj.board.board for state in CLOSED)):
                # If the OPEN list is empty, we will append the first child
                if not OPEN:
                    OPEN.append(oneStateObj)
                else:    
                    # Checking if the current successor board is already present in the OPEN list
                    # If it is and the cost of its "clone" is greater, then we will replace it with the current successor
                    if any((state.board.board == oneStateObj.board.board and state.cost > oneStateObj.cost) for state in OPEN):
                        OPEN.remove(state)
                        OPEN.append(oneStateObj)
                    # If it is a new successor, we append it to the list   
                    else:
                        OPEN.append(oneStateObj)
        
        # Sort this list by cost (priority by cost)
        OPEN.sort(key=lambda x: x.cost, reverse=False)
        

In [13]:
start_time = time.time()
ourFirstTest= UniformCostSearch(Boards[0])
print("Runtime: ", time.time() - start_time)
print(ourFirstTest)

CLOSED List: 


Puzzle Line:  BBIJ....IJCC..IAAMGDDK.MGH.KL.GHFFL.
Cost:  0
None


Puzzle Line:  BBIJ....IJCCG.IAAMGDDK.MGH.KL..HFFL. G99
Cost:  1
None


Puzzle Line:  BBIJ..G.IJCCG.IAAMGDDK.M.H.KL..HFFL. G98
Cost:  1
None


Puzzle Line:  BBIJ....IJCC..IAAMGDDKLMGH.KL.GHFF.. L99
Cost:  1
None


Puzzle Line:  BBIJ....IJCC..IAA.GDDK.MGH.KLMGHFFL. M99
Cost:  1
None


Puzzle Line:  BBIJ....IJCC..IAA.GDDK..GH.KLMGHFFLM M98
Cost:  1
None


Puzzle Line:  BBIJ..G.IJCCG.IAAMGDDK.M.H.KL..HFFL. G98
Cost:  2
None


Puzzle Line:  BBIJ....IJCCG.IAAMGDDKLMGH.KL..HFF.. G99 L99
Cost:  2
None


Puzzle Line:  BBIJ....IJCCG.IAA.GDDK.MGH.KLM.HFFL. G99 M99
Cost:  2
None


Puzzle Line:  BBIJ....IJCCG.IAA.GDDK..GH.KLM.HFFLM G99 M98
Cost:  2
None


Puzzle Line:  BBIJ..G.IJCCG.IAAMGDDKLM.H.KL..HFF.. G98 L99
Cost:  2
None


Puzzle Line:  BBIJ..G.IJCCG.IAA.GDDK.M.H.KLM.HFFL. G98 M99
Cost:  2
None


Puzzle Line:  BBIJ..G.IJCCG.IAA.GDDK...H.KLM.HFFLM G98 M98
Cost:  2
None


Puzzle Line:  BBIJ....IJCC..IAAMGDDKLMGH.